In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
import zipfile
zip_file_path = '/content/drive/MyDrive/wail/PlantVillage.zip'
extract_folder = '/content/sample_data/PlantVillage'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all contents into the specified folder
    zip_ref.extractall(extract_folder)

print(f"Successfully extracted contents to {extract_folder}")

Successfully extracted contents to /content/sample_data/PlantVillage


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import cv2
import os

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
img_size = (224,224)
data_path = "/content/sample_data/PlantVillage/PlantVillage"

In [ ]:
classes = os.listdir(os.path.join(data_path,'train'))

In [ ]:
classes

['Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Strawberry___Leaf_scorch',
 'Corn_(maize)___Common_rust_',
 'Grape___Black_rot',
 'background',
 'Peach___Bacterial_spot',
 'Apple___Apple_scab',
 'Potato___Late_blight',
 'Cherry_(including_sour)___Powdery_mildew',
 'Tomato___Tomato_mosaic_virus',
 'Cherry_(including_sour)___healthy',
 'Pepper,_bell___healthy',
 'Apple___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Blueberry___healthy',
 'Tomato___Late_blight',
 'Peach___healthy',
 'Apple___Black_rot',
 'Soybean___healthy',
 'Tomato___healthy',
 'Grape___Esca_(Black_Measles)',
 'Apple___Cedar_apple_rust',
 'Tomato___Early_blight',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Squash___Powdery_mildew',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Tomato___Bacterial_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Raspberry___healthy',
 'Tomato___Leaf_Mold',
 'Potato___Early_blight',
 'Tomato___Septoria_le

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 500

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(data_path, "train"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(data_path, "val"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes
)

Found 44016 images belonging to 39 classes.
Found 11004 images belonging to 39 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard


log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = GlobalAveragePooling2D()(base_model.output)

model = Model(inputs=base_model.input, outputs=x)


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.layers import Dense

num_classes = len(classes)

x = Dense(256, activation='relu')(model.output)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=model.input, outputs=predictions)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
loaded_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[tb_callback]
)

Epoch 1/20
42/42 [==============================] - 195s 5s/step - loss: 0.5355 - accuracy: 0.8337 - val_loss: 0.6812 - val_accuracy: 0.7927
Epoch 2/20
42/42 [==============================] - 192s 5s/step - loss: 0.5352 - accuracy: 0.8339 - val_loss: 0.6885 - val_accuracy: 0.7893
Epoch 3/20
42/42 [==============================] - 188s 4s/step - loss: 0.5434 - accuracy: 0.8316 - val_loss: 0.6743 - val_accuracy: 0.7937
Epoch 4/20
42/42 [==============================] - 191s 5s/step - loss: 0.5380 - accuracy: 0.8322 - val_loss: 0.6806 - val_accuracy: 0.7896
Epoch 5/20
42/42 [==============================] - 191s 5s/step - loss: 0.5344 - accuracy: 0.8340 - val_loss: 0.6759 - val_accuracy: 0.7933
Epoch 6/20
42/42 [==============================] - 188s 4s/step - loss: 0.5377 - accuracy: 0.8325 - val_loss: 0.6981 - val_accuracy: 0.7810
Epoch 7/20
42/42 [==============================] - 192s 5s/step - loss: 0.5428 - accuracy: 0.8306 - val_loss: 0.6994 - val_accuracy: 0.7850
Epoch 8/20
42

In [ ]:
model.save("PDResnet50-ep60.keras")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save model to Google Drive
loaded_model.save('/content/drive/MyDrive/wail/PDResnet50-ep230.keras')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/wail/PDResnet50-ep230.keras')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
image = plt.imread("download.jpeg")
plt.imshow(image)

FileNotFoundError: ignored

In [ ]:
images = []

In [ ]:
image = load_img("download.jpeg", target_size=(224,224))
image = img_to_array(image)/255.0
images.append(image)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

# Load and preprocess a single image
img_path = 'download (1).jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
# img_array = preprocess_input(img_array)


In [ ]:
test_loss, test_accuracy = loaded_model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


 1/22 [>.............................] - ETA: 40:56 - loss: 13.8364 - accuracy: 0.0039

KeyboardInterrupt: ignored

In [ ]:
test_images, test_labels = test_generator.next()

In [ ]:
predictions = loaded_model.predict(test_images)

16/16 [==============================] - 96s 6s/step


In [ ]:
predictions

array([[1.0804293e-06, 1.9174306e-04, 7.8176236e-06, ..., 3.1534050e-02,
        2.2489025e-04, 1.4385186e-07],
       [9.8316480e-07, 4.9710510e-05, 5.8944488e-16, ..., 1.7740585e-13,
        2.7403850e-04, 1.1149065e-07],
       [1.9878485e-06, 3.0449536e-04, 5.8213043e-08, ..., 1.6678531e-07,
        1.7025402e-05, 7.1164401e-13],
       ...,
       [6.5892798e-01, 7.2233648e-05, 4.1918433e-04, ..., 2.6613919e-04,
        8.2399583e-06, 1.0129833e-05],
       [1.4007021e-06, 6.8385441e-07, 4.0248335e-07, ..., 4.6567734e-06,
        1.1182936e-05, 2.3558645e-05],
       [1.5161163e-04, 3.4736065e-06, 7.1461422e-08, ..., 6.6851675e-09,
        7.9980502e-08, 2.0622504e-15]], dtype=float32)

In [ ]:
yhat = np.argmax(predictions, axis=1).tolist()
ytrue = np.argmax(test_labels, axis=1).tolist()

In [ ]:
classes[np.argmax(predictions[0])]

'Pepper,_bell___Bacterial_spot'

In [ ]:
test_images, test_labels = test_generator.next()

predictions = loaded_model.predict(test_images)

yhat = np.argmax(predictions, axis=1).tolist()
ytrue = np.argmax(test_labels, axis=1).tolist()

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
multilabel_confusion_matrix(ytrue, yhat)

NameError: ignored

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(ytrue, yhat)
cm

array([[0, 0, 4, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
cr = classification_report(ytrue, yhat)
print(cr)

NameError: ignored

In [ ]:
import pandas as pd

report = classification_report(ytrue, yhat, target_names=[f'Class {i}' for i in range(len(classes))])
print(report)

# Convert the classification report to a Pandas DataFrame for better visualization
report_dict = classification_report(ytrue, yhat, target_names=[f'Class {i}' for i in range(len(classes))], output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
print(report_df)

NameError: ignored

In [ ]:
print("Classification Report:\n", classification_report(ytrue, yhat))

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Display the confusion matrix using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(39), yticklabels=range(39))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
classes[20]

'Tomato___Tomato_Yellow_Leaf_Curl_Virus'